In [3]:
import pandas as pd
import geopandas as gpd

In [40]:
# only keep cycleways

In [106]:
gdf = gpd.read_file('/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run1/berlin_ways_pred.shp', dtype={'ID': int}, crs=25833)

In [107]:

# Define the custom order for the ROAD_TYPE column
road_type_order = ['cycleway_hochbord', 'cycleway_lane', 'road', 'pedestrian', 'other']
# Convert the ROAD_TYPE column to a categorical type with the specified order
gdf['ROAD_TYPE'] = pd.Categorical(gdf['ROAD_TYPE'], categories=road_type_order, ordered=True)
# Sort the DataFrame by ID and the custom ordered ROAD_TYPE
gdf.sort_values(['ID', 'ROAD_TYPE'], inplace=True)
gdf['ROAD_TYPE'] = gdf['ROAD_TYPE'].astype(str)

In [109]:
gdf = gdf.groupby('ID').first().reset_index()

In [111]:
gdf.crs = 25833

In [112]:
gdf['ID'] = gdf['ID'].astype(int)

In [113]:
gdf.to_file('/Users/alexandra/Documents/GitHub/road_network_classification/data/berlin_prio_vset/run1/berlin_ways_pred_cycleways.shp')